In [1]:
from diffusers import KandinskyV22Pipeline, KandinskyV22PriorEmb2EmbPipeline, KandinskyV22PriorPipeline
from diffusers.utils import load_image
import torch

pipe_prior = KandinskyV22PriorEmb2EmbPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-prior", torch_dtype=torch.float16
)

pipe_prior.to("cuda:0")

pipe = KandinskyV22Pipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float16
)

pipe.to("cuda:0")


/Users/abhiram/miniconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KandinskyV22Pipeline {
  "_class_name": "KandinskyV22Pipeline",
  "_diffusers_version": "0.18.2",
  "movq": [
    "diffusers",
    "VQModel"
  ],
  "scheduler": [
    "diffusers",
    "DDPMScheduler"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ]
}

In [2]:
prompt = ""
source_img1 = load_image(
    "../assets/dog2.png"  # provide the path to the image
)
source_img2 = load_image(
    "../assets/dog2_with_toy.png"  # provide the path to the image
)

In [3]:
source_img1_feats = pipe_prior._encode_image([source_img1], device="cuda:0", num_images_per_prompt=1)
source_img2_feats = pipe_prior._encode_image([source_img2], device="cuda:0", num_images_per_prompt=1)

# get the edit direction --> edit = feat(image2) - feat(image1)
edit_direction = source_img2_feats - source_img1_feats

In [6]:
target_img = load_image(
    "../assets/cat1.png"  # provide the path to the image
)


In [7]:
target_image_emb, target_negative_image_emb = pipe_prior(prompt, image=target_img, strength=0.1).to_tuple()

python(17615) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
100%|██████████| 2/2 [00:11<00:00,  5.75s/it]


In [8]:
target_image_emb = target_image_emb + edit_direction

In [ ]:
image = pipe(
    image_embeds=target_image_emb,
    negative_image_embeds=target_negative_image_emb,
    height=768,
    width=768,
    num_inference_steps=100,
).images



In [10]:
image[0].save("../data/tests/test.png")